In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [194]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 21.5 MB/s eta 0:00:00


In [248]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup


In [249]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [250]:
CFG = {
    'VIDEO_LENGTH':50, # 10프레임 * 5초
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LR':3e-4,
    'BATCH_SIZE':4,
    'SEED':41,
    'NUM_WORKERS' : 4
}

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 1
frames_per_second = 30
slowfast_alpha = 4
num_clips = 10
num_crops = 3

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [251]:
# colab
df = pd.read_csv(colab_path + '/train.csv')

In [252]:
df['video_path'] = colab_path + df['video_path'].str[1:]

In [253]:
df

,sample_id,video_path,label
0,TRAIN_0000,/content/drive/MyDrive/ColabNotebooks/carcrash...,7
1,TRAIN_0001,/content/drive/MyDrive/ColabNotebooks/carcrash...,7
2,TRAIN_0002,/content/drive/MyDrive/ColabNotebooks/carcrash...,0
3,TRAIN_0003,/content/drive/MyDrive/ColabNotebooks/carcrash...,0
4,TRAIN_0004,/content/drive/MyDrive/ColabNotebooks/carcrash...,1
...,...,...,...
2693,TRAIN_2693,/content/drive/MyDrive/ColabNotebooks/carcrash...,3
2694,TRAIN_2694,/content/drive/MyDrive/ColabNotebooks/carcrash...,5
2695,TRAIN_2695,/content/drive/MyDrive/ColabNotebooks/carcrash...,0
2696,TRAIN_2696,/content/drive/MyDrive/ColabNotebooks/carcrash...,0


In [254]:
class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

In [255]:
class CustomDataset(Dataset):
    def __init__(self, file,device,transform=None, train=True):
        super().__init__()
        self.file = file
        self.len = len(self.file)
        self.device = device
        self.transform = transform
        self.train = train
        self.datalayer = PackPathway()
    
    def __getitem__(self, idx):
        if self.train :
            path = self.file[idx][0]
            label = self.file[idx][1]
            video = EncodedVideo.from_path(path)
            video_data = video.get_clip(start_sec=0, end_sec=5)
            video_data = self.transform(video_data)
            inputs = video_data["video"]
            inputs = [i.to(device) for i in inputs]

            return inputs, label
        else :
            path = self.file[idx]
            video = EncodedVideo.from_path(path)
            video_data = video.get_clip(start_sec=0, end_sec=5)
            video_data = self.transform(video_data)
            inputs = video_data["video"]
            inputs = [i.to(device) for i in inputs]

            return inputs
            

    def __len__(self):
        return self.len

In [256]:
transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


In [257]:
!pip install 'git+https://github.com/facebookresearch/fvcore'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/fvcore to /tmp/pip-req-build-p5rpxej2
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore /tmp/pip-req-build-p5rpxej2
  Resolved https://github.com/facebookresearch/fvcore to commit 51092b5515cbb493f73de079743dd6b11cc4bbf1
  Preparing metadata (setup.py) ... done


In [258]:
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)
model.to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


Net(
  (blocks): ModuleList(
    (0): MultiPathWayWithFuse(
      (multipathway_blocks): ModuleList(
        (0): ResNetBasicStem(
          (conv): Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
          (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
        (1): ResNetBasicStem(
          (conv): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
          (norm): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activation): ReLU()
          (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
        )
      )
      (multipathway_fusion): FuseFastToSlow(
        (conv_fast_to_slow): Conv3d(8, 16, kernel_size=(7, 1, 1), st

In [259]:
# colab
df = pd.read_csv(colab_path + '/train.csv')
df['video_path'] = colab_path + df['video_path'].str[1:]

test = pd.read_csv(colab_path + '/test.csv')
test['video_path'] = colab_path + test['video_path'].str[1:]


In [260]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])


In [264]:
train_data = []
test_data = []

for vid, path, label in train.values :
    train_data.append((path, label))

for vid, path in test.values :
    test_data.append(path)
    
train_dataset = CustomDataset(train_data,device,transform,train=True)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = CustomDataset(test_data,device,transform,train=False)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [265]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(
        nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(
        nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters,
                    lr=5e-5, correct_bias=False)
criterion = nn.CrossEntropyLoss()

In [266]:
for epoch in range(1,10) :
    total_loss = 0
    model.train()
    print("------------TRAIN------------")
    for i, d in enumerate(train_loader): 
        data, label = d
        label = label.to(device)
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output,label)
        
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    print("EPOCH:", epoch)
    print("train_loss:{:.6f}".format(total_loss/len(train_loader)))   



------------TRAIN------------


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), 'set_seed_slowfast.pth')


In [ ]:
model.load_state_dict(torch.load('set_seed_slowfast.pth'))


In [ ]:
total_pred = np.zeros(0)
model.eval()
with torch.no_grad():
    for idx, d in enumerate(test_loader):
        output = model(d)
        
        pred = output.argmax(dim=1, keepdim=True)
        
        total_pred = np.append(total_pred, pred.cpu().numpy())

In [ ]:
total_pred

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit

In [ ]:
submit['label'] = total_pred
submit.head()

In [ ]:
submit.to_csv('set_seed_slowfast_submit.csv', index=False)